In [1]:
import bqplot as bq
import ipywidgets as widgets
import numpy as np
import pandas as pd

In [27]:
def import_image(image_path):
    """Import the image of the plot"""
    with open(image_path, 'rb') as fid:
        raw_img = fid.read()

    ipyimg = widgets.Image(value=raw_img, format=image_path[-3:])
    return ipyimg


def set_image_dim(x_dim, y_dim):
    """Set the scale of the plot, i.e. min/max x, min/max y"""

    scales = {'x': bq.LinearScale(min=np.min(x_dim), max=np.max(x_dim)),
              'y': bq.LinearScale(min=np.min(y_dim), max=np.max(y_dim))}

    return scales


def add_del(scale, img):
    """Add function for ipywidget buttons to add and delete points"""

    scat = bq.Scatter(x=[], y=[], scales=scale, colors=['black'],
                      enable_move=True)

    image = bq.Image(x=np.array([scale['x'].min, scale['x'].max]),
                     y=np.array([scale['y'].min, scale['y'].max]),
                     image=img, scales=scale, enable_hover=False)

    interact_control = widgets.ToggleButtons(options=['Add', 'Delete'],
                                             style={'button_width': '130px',
                                                   'button_style': 'warning'})

    def change_interact(shape):
        """Give a meaning to the buttons."""
        interact_params = {
            'Add': {'interactions': {'click': 'add'},
                    'enable_move': True},
            'Delete': {'interactions': {'click': 'delete'},
                       'enable_move': False}
        }

        for param, value in interact_params[interact_control.value].items():
            setattr(scat, param, value)
    interact_control.observe(change_interact)

    fig = bq.Figure(title='Plot Temp vs Powerdensity', marks=[image, scat],
                    padding_x=0, padding_y=0)
    fig.axes = [bq.Axis(scale=scale['x']), bq.Axis(scale=scale['y'],
                                                   orientation='vertical')]

    return widgets.VBox([fig, interact_control])
    # return widgets.HBox([widgets.VBox([fig, interact_control]),
    #                    print(scat.x, scat.y)])

def write_data(fig, profile_direction: str = 'x', formation: str = 'basement') -> pd.DataFrame:
    """Write picked data to a dataframe

    Args:
        fig (widget): widget object with coordinate data
        profile_direction (string, optional): direction of profile. If 'x', 'y' will be set
        to 1, if 'y', 'x' will be set to one. Defaults to 'x'.
        formation (string, optional): Name of picked formation. Defaults to 'basement'.
    """
    
    x = fig.children[0].marks[1].x
    y = fig.children[0].marks[1].y
        
    df_dict = {'Temp[Celsius]':x, 'PowerDens [MW/sqkm]':y}
    df = pd.DataFrame.from_dict(df_dict)
    
    return df

## Import the Image and set its dimensions

In [28]:
img = import_image('../figures/wilmarth_2019_noborders.PNG')

In [29]:
xdim = np.array([75., 350.]) # temperature from 75 °C to 350 °C
ydim = np.array([0., 45.]) # PowerDensity in MW per square km
dimensions = set_image_dim(xdim, ydim)

In [30]:
# create figure object
click_fig = add_del(dimensions, img)

In [31]:
click_fig

In [36]:
img_output = write_data(click_fig)

In [59]:
lowerPD, higherPD = np.array([img_output['PowerDens [MW/sqkm]'].argmin(),
                     img_output['PowerDens [MW/sqkm]'].argmax()])

# Hannah's implementation

In [60]:
# USER INPUT REQUIRED
#Cumulative confidence of optimistic case
Opt_case = 0.1

In [62]:
# USER INPUT REQUIRED
Tmax_P90 = np.mean(img_output['Temp[Celsius]']) #startup averages temperature for P90 reserves (degrees C)
Tmin = 250 #minimum temperature for the P10 reservoir (degrees C)
print("Startup temperature P90 {:.2f} °C, \nMin temperature for P10 {:.2f} °C".format(*[Tmax_P90,Tmin]))

Startup temperature P90 222.56 °C, 
Min temperature for P10 250.00 °C


In [48]:
# USER INPUT REQUIRED
# Power Density 250 to 280 deg C (MWe/km2)
PowerDens_P90 = img_output['PowerDens [MW/sqkm]'].iloc[lowerPD]
PowerDens_P10 = img_output['PowerDens [MW/sqkm]'].iloc[higherPD]

In [50]:
# USER INPUT REQUIRED
# Area > 250 deg C in km2
Area_P90 = 1
Area_P10 = 10

3.672858973911829